In [11]:
import deepxde as dde
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [12]:
# Set random seed
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)
dde.backend.tf.random.set_random_seed(seed)

# Set hyperparameters
n_output = 4 # postition (x), theta 1, theta 2, force on cart (u_norm)

num_domain = 1000

n_adam = 5000

lr = 2e-2 # for Adam
loss_weights = [1., 10., 1., 1., 1., 1., 1., 1., 1.]

# Set physical parameters
tmin, tmax = 0.0, 10.0
xmin, xmax = -5.0, 5.0
force_max = 1
target = -1. # both theta1 and theta2 should be close to 180 degrees

# Define constants
m1, m2, mc, L1, L2, LC1, LC2, I1, I2, g = [0.1, 0.1, 1, 0.5, 0.5, 0.25, 0.25, 0.01, 0.01, 9.81]  # Example values
Bc, B1, B2 = [0.5, 0.001, 0.001]  # Damping coefficients
u_max = 1  # Maximum force

In [13]:
class Custom_BC(dde.icbc.BC):
    def __init__(self, geom, func, on_boundary, component=0):
        super().__init__(geom, on_boundary, component)
        self.func = dde.icbc.boundary_conditions.npfunc_range_autocache(dde.utils.return_tensor(func))
        
    def error(self, X, inputs, outputs, beg, end, aux_var=None):
        # beg and end specify the current batch range
        values = self.func(X, beg, end, aux_var)
        theta1 = outputs[:, 1:2]
        theta2 = outputs[:, 2:3]
        goal1 = tf.cos(theta1)
        goal2 = tf.cos(theta2)

        return ((goal1[beg:end, self.component:self.component + 1] - values) ** 2) + ((goal2[beg:end, self.component:self.component + 1] - values) ** 2)

In [14]:
def ode_double_pendulum(t, u):
    # Unpack the input tensor
    x, q1, q2, u_norm = u[:, 0:1], u[:, 1:2], u[:, 2:3], u[:, 3:4]
    u_force = u_max * tf.tanh(u_norm)  # Scale and bound the control input

    # Compute time derivatives
    xdot_t, q1dot_t, q2dot_t = [dde.grad.jacobian(var, t) for var in [x, q1, q2]]
    xdot_tt, q1dot_tt, q2dot_tt = [dde.grad.jacobian(var_t, t) for var_t in [xdot_t, q1dot_t, q2dot_t]]

    # Compute intermediates
    h = [tf.reshape(var, (-1, 1)) for var in [mc + m1 + m2, m1 * LC1 + m2 * L1, m2 * LC2, m1 * LC1**2 + m2 * L1**2 + I1, m2 * LC2 * L1, m2 * LC2**2 + I2, m1 * LC1 * g + m2 * L1 * g, m2 * LC2 * g]]
    B_reshaped = [tf.reshape(B, (-1, 1)) for B in [Bc, B1, B2]]

    # Compute matrices
    M = tf.stack([[h[0], h[1] * tf.cos(q1), h[2] * tf.cos(q2)],
                  [h[1] * tf.cos(q1), h[3], h[4] * tf.cos(q1 - q2)],
                  [h[2] * tf.cos(q2), h[4] * tf.cos(q1 - q2), h[5]]], axis=1)

    zero_tensor = tf.zeros_like(B_reshaped[0])
    C = tf.stack([[B_reshaped[0], -h[1] * q1dot_t * tf.sin(q1), -h[2] * q2dot_t * tf.sin(q2)],
                  [zero_tensor, B_reshaped[1] + B_reshaped[2], h[4] * q2dot_t * tf.sin(q1 - q2) - B_reshaped[2]],
                  [zero_tensor, -h[4] * q1dot_t * tf.sin(q1 - q2) - B_reshaped[2], B_reshaped[2]]], axis=1)

    G = tf.stack([tf.zeros_like(h[6]), -h[6] * tf.sin(q1), -h[7] * tf.sin(q2)], axis=0)
    U = tf.stack([u_force, tf.zeros_like(u_force), tf.zeros_like(u_force)], axis=1)

    # Compute DQ, CDQ, b, and Mb
    DQ = tf.expand_dims(tf.stack([xdot_t, q1dot_t, q2dot_t], axis=0), axis=-1)
    CDQ = tf.matmul(C, DQ)
    b = tf.expand_dims(tf.stack([xdot_tt, q1dot_tt, q2dot_tt], axis=0), axis=-1)
    Mb = tf.matmul(M, b)

    # Compute residuals
    residual = Mb - (U - CDQ - G)
    return residual

In [15]:
def initial(_, on_initial):
    return on_initial

def boundary_left(t, on_boundary):
    # on_boundary is passed here by deepxde and it serves as an intial filter
    # which tells if a point lies on the boundary or not

    return on_boundary * np.isclose(t[0], tmin) # np.isclose(t[0], tmin) checks if the point is on
                                                # the left boundary or not and this is a second filter

def boundary_right(t, on_boundary):
    # on_boundary is passed here by deepxde and it serves as an intial filter
    # which tells if a point lies on the boundary or not
    return on_boundary * np.isclose(t[0], tmax) # np.isclose(t[0], tmax) checks if the point is on
                                                # the right boundary or not and this is a second filter

In [16]:
geom = dde.geometry.TimeDomain(tmin, tmax)

ic1 = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=0) # posittion = 0 at time = 0
ic2 = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=1) # theta1 = 0 at time = 0
ic3 = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=2) # theta2 = 0 at time = 0
ic4 = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=3) # force = 0 at time = 0

ic5 = dde.icbc.NeumannBC(geom, lambda t: np.array([0.]), boundary_left, component=0) # cart velocity 1 = 0 at time = 0
ic6 = dde.icbc.NeumannBC(geom, lambda t: np.array([0.]), boundary_left, component=1) # angular velocity 1 = 0 at time = 0
ic7 = dde.icbc.NeumannBC(geom, lambda t: np.array([0.]), boundary_left, component=2) # angular velocity 2 = 0 at time = 0

opt = Custom_BC(geom, lambda t: np.array([target]), boundary_right) # custom ICBC

data = dde.data.PDE(geom, ode_double_pendulum, [ic1, ic2, ic3, ic4, ic5, ic6, ic7, opt], num_domain=num_domain, num_boundary=2)
# dataset size here will be 1002 (1000 domain + 2 boundary)

In [17]:
net = dde.nn.FNN([1] + [64] * 3 + [n_output], "tanh", "Glorot normal")

In [18]:
resampler = dde.callbacks.PDEPointResampler(period=100)

In [19]:
model = dde.Model(data, net)
model.compile("adam", lr=lr, loss_weights=loss_weights)


Compiling model...
Building feed-forward neural network...
'build' took 0.076153 s



/Users/arjundosajh/miniconda3/envs/ml/lib/python3.11/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 1.037483 s



In [20]:
losshistory, train_state = model.train(display_every=100, iterations=n_adam)

Training model...



InvalidArgumentError: Graph execution error:

Detected at node 'gradients_18/strided_slice_37_grad/StridedSliceGrad' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/Users/arjundosajh/miniconda3/envs/ml/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    File "/Users/arjundosajh/miniconda3/envs/ml/lib/python3.11/site-packages/traitlets/config/application.py", line 1077, in launch_instance
    File "/Users/arjundosajh/miniconda3/envs/ml/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 737, in start
    File "/Users/arjundosajh/miniconda3/envs/ml/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
    File "/Users/arjundosajh/miniconda3/envs/ml/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
    File "/Users/arjundosajh/miniconda3/envs/ml/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once
   

In [26]:
a1,a2,a3,a4 = [1,2,3]

ValueError: not enough values to unpack (expected 4, got 3)

In [27]:
t = tf.convert_to_tensor([[0.1]])
u = tf.convert_to_tensor([[0.2, 0.3], [0.2, 0.3], [0.2, 0.3], [0.2, 0.3]])

print(f"Value of residual is: {ode_double_pendulum(t, u)}")

ValueError: i=0 is not valid.

In [ ]:
# # original ODE function:

# def ode_double_pendulum(t, u):
#     x, q1, q2, u_norm = u[:, 0:1], u[:, 1:2], u[:, 2:3], u[:, 3:4]
#     u_force = u_max * tf.tanh(u_norm)  # Scale and bound the control input

#     # Compute time derivatives
#     xdot_t = dde.grad.jacobian(x, t)
#     q1dot_t = dde.grad.jacobian(q1, t)
#     q2dot_t = dde.grad.jacobian(q2, t)

#     xdot_tt = dde.grad.jacobian(xdot_t, t)
#     q1dot_tt = dde.grad.jacobian(q1dot_t, t)
#     q2dot_tt = dde.grad.jacobian(q2dot_t, t)

#     # Intermediates
#     h1 = mc + m1 + m2
#     h2 = m1 * LC1 + m2 * L1
#     h3 = m2 * LC2
#     h4 = m1 * LC1**2 + m2 * L1**2 + I1
#     h5 = m2 * LC2 * L1
#     h6 = m2 * LC2**2 + I2
#     h7 = m1 * LC1 * g + m2 * L1 * g
#     h8 = m2 * LC2 * g

#     # Matrices
#     M = tf.constant([[h1, h2 * tf.cos(q1), h3 * tf.cos(q2)],
#                      [h2 * tf.cos(q1), h4, h5 * tf.cos(q1 - q2)],
#                      [h3 * tf.cos(q2), h5 * tf.cos(q1 - q2), h6]])

    
#     C = tf.constant([[Bc, -h2 * q1dot_t * tf.sin(q1), -h3 * q2dot_t * tf.sin(q2)],
#                      [0, B1 + B2, h5 * q2dot_t * tf.sin(q1 - q2) - B2],
#                      [0, -h5 * q1dot_t * tf.sin(q1 - q2) - B2, B2]])
    

#     G = tf.constant([0, -h7 * tf.sin(q1), -h8 * tf.sin(q2)])
#     U = tf.constant([u_force, 0, 0])
#     DQ = tf.stack([xdot_t, q1dot_t, q2dot_t], axis=1)

#     CDQ = tf.matmul(C, tf.transpose(DQ))
#     b = tf.stack([xdot_tt, q1dot_tt, q2dot_tt], axis=1)
#     Mb = tf.matmul(M, tf.transpose(b))

#     # Equations of Motion (residuals)
#     residual = Mb - (U - CDQ - G)
#     return residual